In [6]:
import pandas as pd
import os

# CSV 파일들이 저장된 폴더
data_folder = "./ais_data"

# 출발지/도착지 박스 설정
def is_in_departure_box(lat, lon):
    return 37.28 <= lat <= 37.36 and 126.40 <= lon <= 126.50

def is_in_arrival_box(lat, lon):
    return 33.5 <= lat <= 33.6 and 126.5 <= lon <= 126.6

# 출발지/도착지 MMSI 저장
departure_mmsi = set()
arrival_mmsi = set()

# 한 달치 반복 (2020년 12월)
for day in range(1, 31):  # 1일부터 31일까지 (윤년 기준)
    filename = f"Dynamic_202004{day:02d}.csv"
    print(f"처리 중인 파일 : {filename}")
    filepath = os.path.join(data_folder, filename)
    
    if not os.path.exists(filepath):
        continue  # 파일 없으면 넘어감

    df = pd.read_csv(filepath, encoding="cp949", skiprows=2)

    # 출발지 MMSI 찾기
    dep = df[
        (df["위도"] >= 37.3) & (df["위도"] <= 37.6) &
        (df["경도"] >= 126.5) & (df["경도"] <= 126.8)
    ]
    departure_mmsi.update(dep["MMSI"].unique())

    # 도착지 MMSI 찾기
    arr = df[
        (df["위도"] >= 33.4) & (df["위도"] <= 33.6) &
        (df["경도"] >= 126.5) & (df["경도"] <= 126.6)
    ]
    arrival_mmsi.update(arr["MMSI"].unique())

# 출발-도착 모두 포함된 MMSI
target_mmsi = departure_mmsi & arrival_mmsi

처리 중인 파일 : Dynamic_20200401.csv
처리 중인 파일 : Dynamic_20200402.csv
처리 중인 파일 : Dynamic_20200403.csv
처리 중인 파일 : Dynamic_20200404.csv
처리 중인 파일 : Dynamic_20200405.csv
처리 중인 파일 : Dynamic_20200406.csv
처리 중인 파일 : Dynamic_20200407.csv
처리 중인 파일 : Dynamic_20200408.csv
처리 중인 파일 : Dynamic_20200409.csv
처리 중인 파일 : Dynamic_20200410.csv
처리 중인 파일 : Dynamic_20200411.csv
처리 중인 파일 : Dynamic_20200412.csv
처리 중인 파일 : Dynamic_20200413.csv
처리 중인 파일 : Dynamic_20200414.csv
처리 중인 파일 : Dynamic_20200415.csv
처리 중인 파일 : Dynamic_20200416.csv
처리 중인 파일 : Dynamic_20200417.csv
처리 중인 파일 : Dynamic_20200418.csv
처리 중인 파일 : Dynamic_20200419.csv
처리 중인 파일 : Dynamic_20200420.csv
처리 중인 파일 : Dynamic_20200421.csv
처리 중인 파일 : Dynamic_20200422.csv
처리 중인 파일 : Dynamic_20200423.csv
처리 중인 파일 : Dynamic_20200424.csv
처리 중인 파일 : Dynamic_20200425.csv
처리 중인 파일 : Dynamic_20200426.csv
처리 중인 파일 : Dynamic_20200427.csv
처리 중인 파일 : Dynamic_20200428.csv
처리 중인 파일 : Dynamic_20200429.csv
처리 중인 파일 : Dynamic_20200430.csv


In [7]:
# 🎯 최종: 전체 경로 중 해당 MMSI만 수집
route_data = []
print("-----최종 phase 진입-----")
for day in range(1, 31):
    filename = f"Dynamic_202004{day:02d}.csv"
    print(f"처리 중인 파일 : {filename}")
    filepath = os.path.join(data_folder, filename)
    
    if not os.path.exists(filepath):
        continue

    df = pd.read_csv(filepath, encoding="cp949",skiprows=2)
    df_target = df[df["MMSI"].isin(target_mmsi)]
    route_data.append(df_target)

# 하나의 DataFrame으로 합치기
final_df = pd.concat(route_data, ignore_index=True)

print("-----정렬 진행중-----")
# 시계열 정렬
final_df = final_df.sort_values(by=["MMSI", "일시"])
print("-----정렬 완료-----")
# 저장할 경우
final_df.to_csv("incheon_to_jeju_routes_202004.csv", index=False, encoding="cp949")

-----최종 phase 진입-----
처리 중인 파일 : Dynamic_20200401.csv
처리 중인 파일 : Dynamic_20200402.csv
처리 중인 파일 : Dynamic_20200403.csv
처리 중인 파일 : Dynamic_20200404.csv
처리 중인 파일 : Dynamic_20200405.csv
처리 중인 파일 : Dynamic_20200406.csv
처리 중인 파일 : Dynamic_20200407.csv
처리 중인 파일 : Dynamic_20200408.csv
처리 중인 파일 : Dynamic_20200409.csv
처리 중인 파일 : Dynamic_20200410.csv
처리 중인 파일 : Dynamic_20200411.csv
처리 중인 파일 : Dynamic_20200412.csv
처리 중인 파일 : Dynamic_20200413.csv
처리 중인 파일 : Dynamic_20200414.csv
처리 중인 파일 : Dynamic_20200415.csv
처리 중인 파일 : Dynamic_20200416.csv
처리 중인 파일 : Dynamic_20200417.csv
처리 중인 파일 : Dynamic_20200418.csv
처리 중인 파일 : Dynamic_20200419.csv
처리 중인 파일 : Dynamic_20200420.csv
처리 중인 파일 : Dynamic_20200421.csv
처리 중인 파일 : Dynamic_20200422.csv
처리 중인 파일 : Dynamic_20200423.csv
처리 중인 파일 : Dynamic_20200424.csv
처리 중인 파일 : Dynamic_20200425.csv
처리 중인 파일 : Dynamic_20200426.csv
처리 중인 파일 : Dynamic_20200427.csv
처리 중인 파일 : Dynamic_20200428.csv
처리 중인 파일 : Dynamic_20200429.csv
처리 중인 파일 : Dynamic_20200430.csv
-----정렬 진행중-----
-

In [ ]:
## Version 1 - 데이터 손실 최소화버전
import pandas as pd
import os
import folium
import matplotlib.cm as cm
import matplotlib.colors as mcolors

final_df = pd.read_csv("incheon_to_jeju_routes_202004.csv", encoding="cp949")

# 1. 이상치 제거 (행 단위)
valid_df = final_df[
    (final_df["위도"] >= 30) & (final_df["위도"] <= 40) &
    (final_df["경도"] >= 122) & (final_df["경도"] <= 133)
].copy()

# 2. MMSI 리스트 (정상 구간이 있는 MMSI 전부)
mmsi_list = valid_df["MMSI"].unique()
num_mmsi = len(mmsi_list)

# 3. 컬러맵 생성
colormap = cm.get_cmap('tab20', num_mmsi)
mmsi_color_dict = {
    mmsi: mcolors.to_hex(colormap(i))
    for i, mmsi in enumerate(mmsi_list)
}

# 4. 지도 생성
map_center = [36.0, 127.5]  # 한반도 중심
m = folium.Map(location=map_center, zoom_start=6)

# 5. MMSI별로 PolyLine 생성 (정상 구간만)
for mmsi, group in valid_df.groupby("MMSI"):
    group = group.sort_values("일시")
    coords = list(zip(group["위도"], group["경도"]))

    # 포인트가 2개 이상 있어야 선을 그림
    if len(coords) >= 2:
        folium.PolyLine(
            locations=coords,
            color=mmsi_color_dict[mmsi],
            weight=2,
            opacity=0.8,
            popup=f"MMSI: {mmsi}"
        ).add_to(m)

# 6. 지도 저장 (선택)
m.save("mmsi_routes_map_202004.html")

C:\Users\김연범\AppData\Local\Temp\ipykernel_9792\4165112815.py:21: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('tab20', num_mmsi)


In [ ]:
## Version 2 - 데이터 손실을 감수하는, 엄밀한 버전 -> 이상치 경로가 하나라도 있으면 해당 MMSI(선박) 제거 
import folium
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as mcolors
import pandas as pd

# 예시: 앞에서 필터링된 final_df가 있다고 가정
final_df = pd.read_csv("incheon_to_jeju_routes_202004.csv", encoding="cp949")

# 위경도 이상치 포함된 MMSI 제거
# 위경도 기준
invalid_mmsi_by_location = final_df[
    (final_df["위도"] < 30) | (final_df["위도"] > 40) |
    (final_df["경도"] < 122) | (final_df["경도"] > 133)
]["MMSI"].unique().tolist()

# 전체 제거 대상 MMSI
all_invalid_mmsi = set(invalid_mmsi_by_location)

# 이상치 MMSI 전체 제거
final_df = final_df[~final_df["MMSI"].isin(all_invalid_mmsi)].copy()

# 1. MMSI 리스트 (고유 선박 식별자)
mmsi_list = final_df["MMSI"].unique()
num_mmsi = len(mmsi_list)

# 2. 컬러맵 생성 
colormap = cm.get_cmap('tab20', num_mmsi)
mmsi_color_dict = {
    mmsi: mcolors.to_hex(colormap(i))
    for i, mmsi in enumerate(mmsi_list)
}

# 3. 지도 생성
map_center = [36.0, 127.5]  # 중간 지점 대략
m = folium.Map(location=map_center, zoom_start=6)

# 4. MMSI별로 PolyLine 생성
for mmsi, group in final_df.groupby("MMSI"):
    group = group.sort_values("일시")
    coords = list(zip(group["위도"], group["경도"]))
    
    folium.PolyLine(
        locations=coords,
        color=mmsi_color_dict[mmsi],
        weight=2,
        opacity=0.8,
        popup=f"MMSI: {mmsi}"
    ).add_to(m)

# 5. 지도 저장 or 표시
m.save("mmsi_routes_map_202004.html")

C:\Users\김연범\AppData\Local\Temp\ipykernel_9792\2016594452.py:28: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  colormap = cm.get_cmap('tab20', num_mmsi)


In [ ]:
## 편도 경로 추출 파이프라인: 이전 과정에서 html 파일을 통해 추출하고자 하는 경로를 클릭, MMSI를 확인한 후, target_mmsi에 그 MMSI를 넣어준다.
## 해당 MMSI의 인천 -> 제주 편도 경로를 하나씩 파일로 추출해주는 코드
from geopy.distance import geodesic
departure_point = (37.32, 126.44)  # 인천 예시
arrival_point = (33.52, 126.52)   # 제주 예시
radius_km = 5.0  # 출도착 판단 반경

target_mmsi = "QQNQcWMDznFEjccPBIYSVw=="
# 최소화 버전 쓸때,
final_df = valid_df
final_df["일시"] = pd.to_datetime(final_df["일시"])
ship_df = final_df[final_df["MMSI"] == target_mmsi].sort_values("일시").reset_index(drop=True)

# ============================
# 편도 경로 여러 개 추출 로직
# ============================
routes = []
i = 0
while i < len(ship_df):
    # 출발지 여러 번 통과한 경우를 추적
    last_departure_idx = None
    while i < len(ship_df):
        latlon = (ship_df.loc[i, "위도"], ship_df.loc[i, "경도"])
        if geodesic(latlon, departure_point).km < radius_km:
            last_departure_idx = i  # 매번 갱신됨
        if geodesic(latlon, arrival_point).km < radius_km and last_departure_idx is not None:
            route_df = ship_df.loc[last_departure_idx:i+1].copy()
            routes.append(route_df)
            i = i + 1  # 다음부터 계속 탐색
            break
        i += 1
    else:
        break  # 더 이상 도착지를 찾지 못했으면 종료

# ============================
# 시각화
# ============================

# 경로 요약 저장용 리스트
summary_list = []

# 시각화 및 거리/시간 계산
for idx, route_df in enumerate(routes, start=1):
    # 거리 계산
    total_distance = 0.0
    coords = list(zip(route_df["위도"], route_df["경도"]))
    for (lat1, lon1), (lat2, lon2) in zip(coords[:-1], coords[1:]):
        total_distance += geodesic((lat1, lon1), (lat2, lon2)).km

    # 시간 계산
    start_time = route_df["일시"].iloc[0]
    end_time = route_df["일시"].iloc[-1]
    duration = end_time - start_time
    duration_hours = duration.total_seconds() / 3600

    # 요약 정보 저장
    summary_list.append({
        "route_index": idx,
        "start_time": start_time,
        "end_time": end_time,
        "duration_hours": round(duration_hours, 2),
        "total_distance_km": round(total_distance, 2),
        "point_count": len(route_df)
    })

    print(summary_list)

    # 📄 CSV 저장 (전체 경로 데이터)
    route_df.to_csv(f"route_{idx}_202004.csv", index=False, encoding="cp949")
    print("저장 완료")
    
    # 시각화
    m = folium.Map(location=departure_point, zoom_start=6)
    folium.PolyLine(coords, color="blue", weight=4, opacity=0.8).add_to(m)
    folium.Marker(departure_point, tooltip="Departure", icon=folium.Icon(color="green")).add_to(m)
    folium.Marker(arrival_point, tooltip="Arrival", icon=folium.Icon(color="red")).add_to(m)
    m.save(f"route_{idx}_202004.html")

[{'route_index': 1, 'start_time': Timestamp('2020-04-01 18:55:47'), 'end_time': Timestamp('2020-04-02 08:09:23'), 'duration_hours': 13.23, 'total_distance_km': 459.16, 'point_count': 7590}]
저장 완료
[{'route_index': 1, 'start_time': Timestamp('2020-04-01 18:55:47'), 'end_time': Timestamp('2020-04-02 08:09:23'), 'duration_hours': 13.23, 'total_distance_km': 459.16, 'point_count': 7590}, {'route_index': 2, 'start_time': Timestamp('2020-04-03 18:40:35'), 'end_time': Timestamp('2020-04-04 08:17:12'), 'duration_hours': 13.61, 'total_distance_km': 462.82, 'point_count': 8359}]
저장 완료
[{'route_index': 1, 'start_time': Timestamp('2020-04-01 18:55:47'), 'end_time': Timestamp('2020-04-02 08:09:23'), 'duration_hours': 13.23, 'total_distance_km': 459.16, 'point_count': 7590}, {'route_index': 2, 'start_time': Timestamp('2020-04-03 18:40:35'), 'end_time': Timestamp('2020-04-04 08:17:12'), 'duration_hours': 13.61, 'total_distance_km': 462.82, 'point_count': 8359}, {'route_index': 3, 'start_time': Timesta